:orphan:
# Exercise 4 - Hydro sections


**Aim:** The aim of this exercise is to get you comfortable plotting and working with matrix (2-dimensional) data.

**Learning outcomes:** At the end of this lab, you will be able to:

1. Plot 2-dimensional fields with a colorbar
   
2. Loop through a 2-dimensional matrix to perform a calculation on each profile

**Data: work with 
<hr>


In [32]:
import matplotlib as plt
import xarray as xr
import pandas as pd

## Load data in matlab format

```{seealso}
See [scipy.io.loadmat](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html) with install instructions [here](https://scipy.org/install/).

    conda install scipy

```

In [55]:
# Load a matlab data file
# scipy.io.loadmat(file_name, mdict=None, appendmat=True, **kwargs)
if 0:
    import scipy.io
    file_path = '../data/wbts_data.mat'
    data1 = scipy.io.loadmat(file_path)
    
    # Split out the variables
    crs = data1['crs'][0]
    stn = data1['stn'][0]
    lat = data1['lat'][0]
    lon = data1['lon'][0]
    tme_days_since_0000 = data1['tme'][0]
    # But wbts_data which was a Matlab structure is very ugly
    # In Matlab, it was a 1 x 777 array of structures where each structure had the fields:
    # pr (pressure)
    # te (temperature)
    # sa (absolute salinity)
    # dh (dynamic height)
    # ox (oxygen)
    # time (in matlab time), 1x1
    # stn (station number), 1x1
    # lat 1x1
    # lon 1x1
    wbts_data = data1['wbts_data'][0]
    
    for i in range(len(wbts_data)):
        pr = wbts_data[0][0][0:]
    #    te = wbts_data[0][1][0:]
    #    sa = wbts_data[0][2][0:]
    #    dh = wbts_data[0][3][0:]
    #    ox = wbts_data[0][4][0:]
    #    days_since_0000 = wbts_data[0][5][0]
    #    stn = wbts_data[0][6][0]
    #   lon = wbts_data[0][8][0]
    #    lat = wbts_data[0][7][0]

file_path = '../data/wbts_data.nc'
data1 = xr.open_dataset(file_path)

In [56]:
print(data1)

<xarray.Dataset>
Dimensions:   (lon: 29, pressure: 4146)
Coordinates:
  * lon       (lon) float64 -76.73 -76.76 -76.77 -76.76 ... -76.74 -76.75 -76.75
  * pressure  (pressure) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... nan nan nan nan nan
Data variables:
    lat       (lon) float64 ...
    time      (lon) datetime64[ns] ...
    temp      (lon, pressure) float64 ...
    salin     (lon, pressure) float64 ...


In [58]:
# Example without a function to parse parts of the header file.
file_path = '../data/JR310_068.hex'

with open(file_path, 'r') as file:
        lines = file.readlines()

# Find the line with latitude
header_line_index = next((i for i, line in enumerate(lines) if line.startswith('* NMEA Latitude')), None)

# Split the line at spaces
lat_parser = lines[header_line_index].strip().split()[1:]

# Extract the data corresponding to latitude and convert from string to float (decimal number)
lat_degrees = float(lat_parser[3])
lat_minutes = float(lat_parser[4])

# Adjust the sign for degrees  south
if lat_parser[5]=='S':
    lat_degrees = -lat_degrees
    lat_minutes = -lat_minutes


lat = lat_degrees + lat_minutes/60

print(lat)

# Find the line with time
header_line_index = next((i for i, line in enumerate(lines) if line.startswith('* NMEA UTC')), None)

# Split the line at spaces
time_parser = lines[header_line_index].strip().split()[1:]

# Pull out the individual time elements
mo_string = time_parser[4]
day_string = time_parser[5]
year_string = time_parser[6]
HHMMSS_string = time_parser[7]
mydatestr = [year_string + '-' + mo_string + '-' + day_string + ' ' + HHMMSS_string]
# This datestr has the format `2015-Apr-09 19:55:52` 
data = {'date': mydatestr}

#
# More on reading/parsing date and time formats in pandas:
# https://saturncloud.io/blog/how-to-convert-datetime-to-string-with-series-in-pandas/#:~:text=with%20data%20analysis.-,Converting%20datetime%20to%20string%20in%20Pandas,to%20a%20specified%20format%20string.
#
df = pd.DataFrame(data)
# We write the format here to match the format of the data in the hex file.
df['date'] = pd.to_datetime(df['date'], format='%Y-%b-%d %H:%M:%S',errors='coerce')

print(df)

-60.644666666666666
                 date
0 2015-04-09 19:55:52


In [53]:
def read_hex_header(file_path: str, encoding = 'latin-1') -> xr.Dataset:
    ''' Reads a *.HEX file from Seabird into an xarray dataset. '''

    # Read the file
    with open(file_path, 'r', encoding=encoding) as file:
        lines = file.readlines()

    #-------
    # Find the line with latitude
    header_line_index = next((i for i, line in enumerate(lines) if line.startswith('* NMEA Latitude')), None)

    # Split the line at spaces
    lat_parser = lines[header_line_index].strip().split()[1:]

    # Extract the data corresponding to latitude and convert from string to float (decimal number)
    lat_degrees = float(lat_parser[3])
    lat_minutes = float(lat_parser[4])

    # Adjust the sign for degrees  south
    if lat_parser[5]=='S':
        lat_degrees = -lat_degrees
        lat_minutes = -lat_minutes

    # Convert to decimal degrees
    lat = lat_degrees + lat_minutes/60

    #-------
    # Find the line with longitude
    header_line_index = next((i for i, line in enumerate(lines) if line.startswith('* NMEA Longitude')), None)

    # Split the line at spaces
    lon_parser = lines[header_line_index].strip().split()[1:]

    # Extract the data corresponding to latitude and convert from string to float (decimal number)
    lon_degrees = float(lon_parser[3])
    lon_minutes = float(lon_parser[4])

    # Adjust the sign for degrees east
    if lat_parser[5]=='W':
        lon_degrees = -lon_degrees
        lon_minutes = -lon_minutes

    # Convert to decimal degrees
    lon = lon_degrees + lon_minutes/60

    #-------
    # Find the line with time
    header_line_index = next((i for i, line in enumerate(lines) if line.startswith('* NMEA UTC')), None)
    
    # Split the line at spaces
    time_parser = lines[header_line_index].strip().split()[1:]
    
    # Pull out the individual time elements
    mo_string = time_parser[4]
    day_string = time_parser[5]
    year_string = time_parser[6]
    HHMMSS_string = time_parser[7]
    mydatestr = [year_string + '-' + mo_string + '-' + day_string + ' ' + HHMMSS_string]
    # This datestr has the format `2015-Apr-09 19:55:52` 
    data = {'date': mydatestr}
    
    #
    # More on reading/parsing date and time formats in pandas:
    # https://saturncloud.io/blog/how-to-convert-datetime-to-string-with-series-in-pandas/#:~:text=with%20data%20analysis.-,Converting%20datetime%20to%20string%20in%20Pandas,to%20a%20specified%20format%20string.
    #
    df = pd.DataFrame(data)
    # We write the format here to match the format of the data in the hex file.
    df['date'] = pd.to_datetime(df['date'], format='%Y-%b-%d %H:%M:%S',errors='coerce')

    df['lon'] = lon
    df['lat'] = lat

    # Convert DataFrame to xarray dataset
    ds = xr.Dataset.from_dataframe(df.set_index('date'))

    return ds

In [59]:
file_path = '../data/JR310_068.hex'
read_hex_header(file_path)


<xarray.Dataset>
Dimensions:  (date: 1)
Coordinates:
  * date     (date) datetime64[ns] 2015-04-09T19:55:52
Data variables:
    lon      (date) float64 42.09
    lat      (date) float64 -60.64

In [25]:
# Read through all the files to find the locations


## Clone the course repository

```{note}
The instructions below assume you are using GitHub desktop which is a desktop application for interacting with any online git (i.e., you do not need a github account.)  See also [Resources: Git and Python](../resource/git) for installation of GitHub desktop.
```

Once you have the software installed, you will want to first clone the class repository.  The repositories we use will live on the UHH gitlab server: [https://gitlab.rrz.uni-hamburg.de/ifmeo/teaching/IfM_SeaOcean/uhh-seaocean-2024](https://gitlab.rrz.uni-hamburg.de/ifmeo/teaching/IfM_SeaOcean/uhh-seaocean-2024). Within this folder, there are several individual 'repositories'.  You will want to clone each of them.

Each of these is an individual "repository":
1. Part 1 - Basics - [https://gitlab.rrz.uni-hamburg.de/ifmeo/teaching/IfM_SeaOcean/uhh-seaocean-2024/part-1-basics](https://gitlab.rrz.uni-hamburg.de/ifmeo/teaching/IfM_SeaOcean/uhh-seaocean-2024/part-1-basics)

You may need to "request access" to the repository.  

1. Access the **course group gitlab**: [https://gitlab.rrz.uni-hamburg.de/ifmeo/teaching/IfM_SeaOcean/uhh-seaocean-2024/part-1-basics) and select a repository.

2. If you are prompted to request access, then do so.  You will need to log in with your B-kennung.

Then you need to clone the repository.  For this, have a 

1. A browswer window open (edge, chrome, safari, firefox, etc) and navigate to the **group gitlab**.  

2. Github desktop (purple)

Decide where on your computer you want these folders to live.  If you don't specify, the default is something like `Documents\GitHub\part-1-basics`.  You may want to nest it inside a folder that has the slug `seaocn` so you can find these later.

1a. In Github desktop, select "clone a repository", and click the "URL" at the right.

2a.  In the browser window, select the address of the repository you would like to clone first.  Copy this address (starting with https:)

1b. In Github desktop, past the URL of the repository into the first text box.  The second text box will auto-populate with a location on your computer.  If you'd like to change this from default, use the button to the right to pick where these will live.  Click "ok" or "next".  *Authentication will fail.* Kein problem.

2b. In the browswer window on the chosen repository (must match the one you used in the previous step), click on "settings" at the bottom of the list to the left.  Choose "Access tokens".  Near the top right, click "Add new token".  Give it a name (maybe your name).  Choose an expiration date at least 1 month in the future but no more than 12 months into the future.  Select a role "maintainer".  Scroll down over all the choices and click "create project access token".  In the green bar that appears, click the "clipboard" button to copy this token.

1c.  In Github desktop, enter your B-kennung in the top text bar "username", but for "password" paste the access token that you just copied.  Click "next".

You should now have cloned the repository.  Check that the repository (which may be an empty folder) lives on your computer.  

**Repeat this step for the other course repositories.**





# Super-brief explanation of git

Git is keeping track of changes to your files within the local repository (your copy of the course folder on your computer).  When you change a file in the folder, git knows.  How do you know it knows?  If you go into your Github desktop (purple icon, desktop app) and choose the repository you're working in in the dropdown menu on the left, and you have changed a file, the filename  for the file with changes will appear in a list, and to the right you can preview the changes that have been made (previous commit and current edited version).  

After you've been working on a file for a while (a day, an hour, 5 minutes, etc), e.g. a piece of code in your repository on your computer, and you've made some important changes, you will want to tell git to save a snapshot of these changes.  To do this you click **commit** in Github desktop.  This will then make git store a version according to the point in time when you clicked commit.  *Advanced tip: in the commit message, you can write something useful to describe the changes, like "added a new function to load data from a microcat".*  So far, only your own computer knows about these changes.

When you want to share these changes with your group, i.e. have these changes show up in the repository on the gitlab.rrz.uni-hamburg.de location, then you need to **push** the changes.  This is a button to the right in your Github desktop (or top right), where you click.  This will then make the changes appear in the online copy of the repository.

When your group member has made changes, and you want these changes to appear in your copy of the code on your own computer, then you need to **fetch** these changes from the version on gitlab.rrz.uni-hamburg.de.  This will not overwrite any of your files.  It will simply add what they have done.

**Note:** For beginner usage, I recommend that you all keep a separate copy of your python code to start with.  If you work on the same piece of code, it's possible for you to delete something and commit it to your repository (on your own computer), and for someone else to edit the same thing without deleting it and commit it on their own computer.  If you both then push these changes to the repository, or you fetch your colleagues changes from the online repository onto your computer, you will get a **conflict** which needs to be merged manually.

Read more: [https://git-scm.com/book/en/v2/Getting-Started-What-is-Git%3F](https://git-scm.com/book/en/v2/Getting-Started-What-is-Git%3F)

More advanced: It's best to work inside an environment.  This controls the version of each of the packages that you have installed, so that the code is more likely to run smoothly on another person's computer.  Some background on [conda environments](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html).


# Check whether you have python installed

In Mac or Linux, at the command line, type
```
python --version
```

For example, mine says
```
(base) 9:34 ~ $ python --version
Python 3.9.5
```

## Using conda

For Mac users, you will probably want `conda` and `pip` installed on your computer.

You can get conda here: [https://conda.io/projects/conda/en/latest/user-guide/install/index.html](https://conda.io/projects/conda/en/latest/user-guide/install/index.html).  I would recommend *miniconda* (Here is the direct link for [installing miniconda](https://docs.anaconda.com/free/miniconda/miniconda-install/)).  If you already have Anaconda, that is fine too.

We will also use jupyter notebooks in this course.

## Installing jupyterlab

You can install jupyterlab with conda

    conda install -c conda-forge jupyterlab=4.0.7 notebook=7.0.6

Since we recommend managing your environments, then conda (as above) is the better way to install it.  Otherwise you can use pip: [https://jupyter.org/install](https://jupyter.org/install).

If you already have Anaconda, then jupyter lab comes by default [explained here](https://test-jupyter.readthedocs.io/en/latest/install.html).

## Now, open this notebook in jupyter notebook.

Please record for yourself the steps you took to get jupyter running on your computer.

## (optional) setting up an environment

On a Mac, in a terminal window, you will create an environment using a specified version of python.

```
conda create --name seaocn_env python=3.8 -y
conda activate seaocn_env
```
